# Datei- und Verzeichnis-Suchtool für Google Colab

Ein praktisches Projekt zur Anwendung von:
- **Rekursion**: Durchqueren des Verzeichnisbaums
- **Decorator**: Laufzeitmessung der Suchfunktionen
- **Closures**: Erweiterte Suchfunktionalität

## Funktionen
- Rekursive Suche nach Dateien nach Erweiterung
- Suche nach Dateinamen (exakt oder Teilstring)
- Laufzeitmessung mit Decorator
- Flexible Suchoptionen

**Hinweis**: Dieses Notebook ist speziell für Google Colab optimiert und verwendet das `/content` Verzeichnissystem.


In [ ]:
import os
import time
from pathlib import Path
from functools import wraps
from typing import List, Callable, Optional

# Prüfe ob wir auf Google Colab laufen
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colab erkannt - Notebook ist bereit!")
    print(f"📁 Arbeitsverzeichnis: {os.getcwd()}")
except ImportError:
    IN_COLAB = False
    print("⚠️  Warnung: Dieses Notebook ist für Google Colab optimiert!")
    print("   Bitte führen Sie es auf Google Colab aus.")


## Decorator für Laufzeitmessung

Ein Decorator, der die Ausführungszeit einer Funktion misst und anzeigt.


In [ ]:
def laufzeit_messung(func: Callable) -> Callable:
    """
    Decorator zur Messung der Ausführungszeit einer Funktion.
    
    Args:
        func: Die zu messende Funktion
        
    Returns:
        Wrapped Funktion mit Laufzeitmessung
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        laufzeit = end_time - start_time
        print(f"⏱️  Funktion '{func.__name__}' benötigte {laufzeit:.4f} Sekunden")
        return result
    return wrapper


## Rekursive Suchfunktionen

Die Hauptfunktionen zur rekursiven Durchsuchung des Verzeichnisbaums.


In [ ]:
@laufzeit_messung
def suche_dateien_nach_erweiterung(verzeichnis: str, erweiterung: str) -> List[str]:
    """
    Sucht rekursiv nach allen Dateien mit einer bestimmten Erweiterung.
    
    Args:
        verzeichnis: Der Startpfad für die Suche
        erweiterung: Die Dateierweiterung (z.B. '.txt', '.py')
        
    Returns:
        Liste aller gefundenen Dateipfade
    """
    gefundene_dateien = []
    
    # Normalisiere die Erweiterung (füge Punkt hinzu falls nicht vorhanden)
    if not erweiterung.startswith('.'):
        erweiterung = '.' + erweiterung
    
    def _rekursive_suche(pfad: Path):
        """Innere rekursive Funktion (Closure)"""
        try:
            # Durchlaufe alle Einträge im aktuellen Verzeichnis
            for eintrag in pfad.iterdir():
                if eintrag.is_file():
                    # Prüfe ob die Datei die gewünschte Erweiterung hat
                    if eintrag.suffix.lower() == erweiterung.lower():
                        gefundene_dateien.append(str(eintrag))
                elif eintrag.is_dir():
                    # Rekursiver Aufruf für Unterverzeichnisse
                    _rekursive_suche(eintrag)
        except PermissionError:
            # Überspringe Verzeichnisse ohne Zugriffsrechte
            pass
    
    start_pfad = Path(verzeichnis)
    if not start_pfad.exists():
        print(f"⚠️  Warnung: Verzeichnis '{verzeichnis}' existiert nicht!")
        return []
    
    _rekursive_suche(start_pfad)
    return gefundene_dateien


In [ ]:
@laufzeit_messung
def suche_dateien_nach_name(verzeichnis: str, name_pattern: str, exakt: bool = False) -> List[str]:
    """
    Sucht rekursiv nach Dateien, deren Name einem Muster entspricht.
    
    Args:
        verzeichnis: Der Startpfad für die Suche
        name_pattern: Der zu suchende Dateiname oder Teilstring
        exakt: Wenn True, muss der Name exakt übereinstimmen
        
    Returns:
        Liste aller gefundenen Dateipfade
    """
    gefundene_dateien = []
    
    def _rekursive_suche(pfad: Path):
        """Innere rekursive Funktion (Closure)"""
        try:
            for eintrag in pfad.iterdir():
                if eintrag.is_file():
                    dateiname = eintrag.name
                    if exakt:
                        if dateiname == name_pattern:
                            gefundene_dateien.append(str(eintrag))
                    else:
                        if name_pattern.lower() in dateiname.lower():
                            gefundene_dateien.append(str(eintrag))
                elif eintrag.is_dir():
                    _rekursive_suche(eintrag)
        except PermissionError:
            pass
    
    start_pfad = Path(verzeichnis)
    if not start_pfad.exists():
        print(f"⚠️  Warnung: Verzeichnis '{verzeichnis}' existiert nicht!")
        return []
    
    _rekursive_suche(start_pfad)
    return gefundene_dateien


In [ ]:
@laufzeit_messung
def suche_dateien_erweitert(verzeichnis: str, 
                            erweiterung: Optional[str] = None,
                            name_enthält: Optional[str] = None,
                            min_größe: Optional[int] = None,
                            max_größe: Optional[int] = None) -> List[dict]:
    """
    Erweiterte Suchfunktion mit mehreren Filtern.
    
    Args:
        verzeichnis: Der Startpfad für die Suche
        erweiterung: Optional: Dateierweiterung (z.B. '.txt')
        name_enthält: Optional: Teilstring im Dateinamen
        min_größe: Optional: Minimale Dateigröße in Bytes
        max_größe: Optional: Maximale Dateigröße in Bytes
        
    Returns:
        Liste von Dictionaries mit Dateiinformationen
    """
    gefundene_dateien = []
    
    def _rekursive_suche(pfad: Path):
        """Innere rekursive Funktion (Closure)"""
        try:
            for eintrag in pfad.iterdir():
                if eintrag.is_file():
                    # Prüfe alle Filter
                    passt = True
                    
                    if erweiterung:
                        norm_erw = erweiterung if erweiterung.startswith('.') else '.' + erweiterung
                        if eintrag.suffix.lower() != norm_erw.lower():
                            passt = False
                    
                    if passt and name_enthält:
                        if name_enthält.lower() not in eintrag.name.lower():
                            passt = False
                    
                    if passt and (min_größe is not None or max_größe is not None):
                        datei_größe = eintrag.stat().st_size
                        if min_größe is not None and datei_größe < min_größe:
                            passt = False
                        if max_größe is not None and datei_größe > max_größe:
                            passt = False
                    
                    if passt:
                        gefundene_dateien.append({
                            'pfad': str(eintrag),
                            'name': eintrag.name,
                            'größe': eintrag.stat().st_size,
                            'erweiterung': eintrag.suffix
                        })
                        
                elif eintrag.is_dir():
                    _rekursive_suche(eintrag)
        except PermissionError:
            pass
    
    start_pfad = Path(verzeichnis)
    if not start_pfad.exists():
        print(f"⚠️  Warnung: Verzeichnis '{verzeichnis}' existiert nicht!")
        return []
    
    _rekursive_suche(start_pfad)
    return gefundene_dateien


In [ ]:
# Suche nach allen .txt-Dateien im /content Verzeichnis (Google Colab Standard)
aktuelles_verzeichnis = '/content'  # Google Colab Standard-Verzeichnis

txt_dateien = suche_dateien_nach_erweiterung(aktuelles_verzeichnis, '.txt')
print(f"\n📄 Gefundene .txt-Dateien: {len(txt_dateien)}")
for datei in txt_dateien[:10]:  # Zeige maximal 10 Dateien
    print(f"  - {datei}")
if len(txt_dateien) > 10:
    print(f"  ... und {len(txt_dateien) - 10} weitere")


### Beispiel 2: Suche nach Python-Dateien (.py)


In [ ]:
# Suche nach allen Python-Dateien
py_dateien = suche_dateien_nach_erweiterung(aktuelles_verzeichnis, 'py')
print(f"\n🐍 Gefundene Python-Dateien: {len(py_dateien)}")
for datei in py_dateien[:10]:
    print(f"  - {datei}")


### Beispiel 3: Suche nach Dateinamen


In [ ]:
# Suche nach Dateien, die "test" im Namen enthalten
test_dateien = suche_dateien_nach_name(aktuelles_verzeichnis, 'test', exakt=False)
print(f"\n🔍 Gefundene Dateien mit 'test' im Namen: {len(test_dateien)}")
for datei in test_dateien[:10]:
    print(f"  - {datei}")


### Beispiel 4: Erweiterte Suche mit mehreren Filtern


In [ ]:
# Erweiterte Suche: .txt-Dateien, die "readme" im Namen enthalten
# und zwischen 100 Bytes und 1 MB groß sind
ergebnisse = suche_dateien_erweitert(
    aktuelles_verzeichnis,
    erweiterung='.txt',
    name_enthält='readme',
    min_größe=100,
    max_größe=1024*1024  # 1 MB
)

print(f"\n📊 Gefundene Dateien: {len(ergebnisse)}")
for datei_info in ergebnisse[:10]:
    größe_kb = datei_info['größe'] / 1024
    print(f"  - {datei_info['name']} ({größe_kb:.2f} KB)")
    print(f"    Pfad: {datei_info['pfad']}")


### Beispiel 5: Suche in spezifischen Google Colab Verzeichnissen

Durchsuchen Sie spezifische Verzeichnisse in Google Colab:


In [ ]:
# Suche im /content Verzeichnis nach Notebook-Dateien
colab_dateien = suche_dateien_nach_erweiterung('/content', '.ipynb')
print(f"\n📓 Gefundene Notebook-Dateien: {len(colab_dateien)}")
for datei in colab_dateien[:10]:
    print(f"  - {datei}")

# Suche in einem spezifischen Unterverzeichnis (z.B. sample_data)
# colab_dateien = suche_dateien_nach_erweiterung('/content/sample_data', '.csv')


## Zusammenfassung

Dieses Tool demonstriert:

✅ **Rekursion**: Die Funktionen `_rekursive_suche()` durchlaufen rekursiv den gesamten Verzeichnisbaum

✅ **Closures**: Die inneren Funktionen `_rekursive_suche()` sind Closures, die auf die äußeren Variablen (`gefundene_dateien`, Parameter) zugreifen

✅ **Decorator**: Der `@laufzeit_messung` Decorator misst automatisch die Ausführungszeit jeder Suchfunktion

✅ **Praktische Anwendung**: Flexibles Dateisuchsystem mit verschiedenen Filtermöglichkeiten


## Optional: Test-Dateien erstellen (für Demonstration)

Erstellen Sie einige Test-Dateien, um die Funktionalität zu testen:


In [ ]:
# Erstelle Test-Verzeichnisstruktur für Demonstration in Google Colab
import os

test_dir = '/content/test_verzeichnis'  # Verwendung von /content für Google Colab
os.makedirs(test_dir, exist_ok=True)
os.makedirs(f'{test_dir}/unterordner1', exist_ok=True)
os.makedirs(f'{test_dir}/unterordner2', exist_ok=True)

# Erstelle einige Test-Dateien
test_dateien = [
    f'{test_dir}/datei1.txt',
    f'{test_dir}/datei2.txt',
    f'{test_dir}/readme.txt',
    f'{test_dir}/test_datei.py',
    f'{test_dir}/unterordner1/datei3.txt',
    f'{test_dir}/unterordner1/datei4.py',
    f'{test_dir}/unterordner2/readme_test.txt',
]

for datei in test_dateien:
    with open(datei, 'w') as f:
        f.write(f'Inhalt von {datei}')

print("✅ Test-Verzeichnisstruktur erstellt!")
print(f"   Verzeichnis: {test_dir}/")


In [ ]:
# Teste die Suche im Test-Verzeichnis
print("=== Test: Suche nach .txt-Dateien ===")
txt_ergebnisse = suche_dateien_nach_erweiterung(test_dir, '.txt')
print(f"Gefunden: {len(txt_ergebnisse)} Dateien")
for datei in txt_ergebnisse:
    print(f"  ✓ {datei}")

print("\n=== Test: Erweiterte Suche ===")
erweiterte_ergebnisse = suche_dateien_erweitert(
    test_dir,
    erweiterung='.txt',
    name_enthält='readme'
)
print(f"Gefunden: {len(erweiterte_ergebnisse)} Dateien")
for datei_info in erweiterte_ergebnisse:
    print(f"  ✓ {datei_info['name']} - {datei_info['größe']} Bytes")
